# Faucet & Wallet analysis
Let's take an initial look at the faucets and people that donate to them.

In [2]:
# some standard imports...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import web3
import os
import sys
import json
import time
import requests

with open("../secrets.json") as f:
    secrets = json.load(f)
    f.close()

if not os.path.exists("out"):
    os.makedirs("out")

etherscan_key = secrets["etherscan_key"]
alchemy_url = secrets["alchemy_https"]
web3 = web3.Web3(web3.HTTPProvider(alchemy_url))

In [10]:
POLYGONSCAN_API_KEY = secrets["polygonscan_key"]

def get_all_transactions(
    address, net, startblock=0, endblock=999999999, print_url=False
):
    """
    Get all transactions for an address
    https://api.polygonscan.com/api
    ?module=account
    &action=txlist
    &address=0xb91dd8225Db88dE4E3CD7B7eC538677A2c1Be8Cb
    &startblock=0
    &endblock=99999999
    &page=1
    &offset=10
    &sort=asc
    &apikey=YourApiKeyToken

    """
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": startblock,
        "endblock": endblock,
        "sort": "desc",
        "apikey": POLYGONSCAN_API_KEY,
    }
    headers = {"User-Agent": "Chrome/41.0.2228.0"}

    url = "https://api-testnet.polygonscan.com/api"

    if print_url:
        print(url + "?" + "&".join("{}={}".format(k, v) for k, v in params.items()))
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "0":
            print(f"Error: {data}")
        else:
            return data["result"]
    else:
        print("Error:", response.status_code)
        print(response.text)
        return None

In [11]:
# import faucets.py
from faucets import Faucets


print(f"List of faucets on Polygon test nets:")
for faucet_net in Faucets:
    for faucet in Faucets[faucet_net]:
        print(faucet)

List of faucets on Polygon test nets:
0x631E9B031b16b18172a2B9D66C3668A68a668d20
0x7F092e65C688a509737FcD8D0998dD12208f5297
0x7a55f1544d7387E97596F8ebe604a0BfB9b33f37
0x4281ecf07378ee595c564a59048801330f3084ee
0x86652c1301843B4E06fBfbBDaA6849266fb2b5e7


In [12]:
# get all transactions and add them to df
txs_df = pd.DataFrame()
txs_df["net"] = ""
for net in Faucets:
    for faucet in Faucets[net]:
        try:
            print(f"Getting transactions for {faucet} on {net}")
            txs = get_all_transactions(faucet, net, print_url=True)
            temp_df = pd.DataFrame(txs)
            oldest_block = min(temp_df["blockNumber"])
            newest_block = max(temp_df["blockNumber"])
            print(
                f"Got {len(txs)} transactions for {faucet} on {net}, oldest block {oldest_block}, newest block {newest_block}"
            )

            # etherscan has a 10k transaction limit, so we iteratively get them
            # until we get all transactions
            while oldest_block != 0:
                txs = get_all_transactions(
                    faucet, net, endblock=oldest_block, print_url=False
                )
                # append to temp df
                temp_df = pd.concat([temp_df, pd.DataFrame(txs)])
                # update oldest block
                newest_block = max(temp_df["blockNumber"])
                temp_oldest_block = min(temp_df["blockNumber"])
                print(
                    f"Got {len(txs)} transactions for {faucet} on {net}, oldest block {temp_oldest_block}, newest block {newest_block}"
                )
                if temp_oldest_block == oldest_block:
                    break
                oldest_block = temp_oldest_block

            print(
                f"{faucet} on {net}: block {oldest_block} to {newest_block}, {len(temp_df)} transactions"
            )

            temp_df["faucet"] = faucet
            temp_df["net"] = net
            # add to main df
            txs_df = pd.concat([txs_df, temp_df])

            # save
            temp_df.to_csv(f"out/{faucet}-{net}.csv", index=False)
            txs_df.to_csv(f"out/txs_df.csv", index=False)
        except Exception as e:  
            print(e)
            # try again
            print(f"Error getting transactions for {faucet} on {net}, trying again")
            time.sleep(1)
            continue

txs_df["value"] = txs_df["value"].astype(np.float128)

Getting transactions for 0x631E9B031b16b18172a2B9D66C3668A68a668d20 on mumbai
https://api-testnet.polygonscan.com/api?module=account&action=txlist&address=0x631E9B031b16b18172a2B9D66C3668A68a668d20&startblock=0&endblock=999999999&sort=desc&apikey=U9J5DQTN6CQIXSDHCD8MSYDCPJADS2JBBY
Got 10000 transactions for 0x631E9B031b16b18172a2B9D66C3668A68a668d20 on mumbai, oldest block 26635312, newest block 27477762
Got 6854 transactions for 0x631E9B031b16b18172a2B9D66C3668A68a668d20 on mumbai, oldest block 25412451, newest block 27477762
Got 1 transactions for 0x631E9B031b16b18172a2B9D66C3668A68a668d20 on mumbai, oldest block 25412451, newest block 27477762
0x631E9B031b16b18172a2B9D66C3668A68a668d20 on mumbai: block 25412451 to 27477762, 16855 transactions
Getting transactions for 0x7F092e65C688a509737FcD8D0998dD12208f5297 on mumbai
https://api-testnet.polygonscan.com/api?module=account&action=txlist&address=0x7F092e65C688a509737FcD8D0998dD12208f5297&startblock=0&endblock=999999999&sort=desc&apik

In [5]:
# save temp_df to sqlite
import sqlite3

conn = sqlite3.connect("out/txs_db.sqlite")
temp_df.to_sql("txs", conn, if_exists="replace", index=False)

22155

In [6]:
temp_df.iloc[0]

blockNumber                                                    1503808
timeStamp                                                   1657796400
hash                 0xcb1f4e9c63e68b90c466f4fbb69bb968fed73f7578bc...
nonce                                                              163
blockHash            0xf70af7678b783da911e92adda938ba587eaa3263c8b0...
transactionIndex                                                     1
from                        0x7495dee08f1340fa1fbda59c979f799d7d676fce
to                          0xb2a8e24a90e5b5f7f4cbd26d350b83674652d65e
value                                                             7495
gas                                                              21000
gasPrice                                                    1500000007
isError                                                              0
txreceipt_status                                                     1
input                                                               0x
contra

In [7]:
temp_df.describe()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,faucet,net
count,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155,22155
unique,8362,8362,22143,22130,8362,140,9,7936,8,1,60,1,1,1,1,917,1,8362,1,1
top,1330334,1655545303,0xbde5fb48cf1c4e548202e734fbd9f2bf02a1995e83e0...,0,0x9cc10918ee5f568310eaa1246506acd2cb45a5ab8d7f...,0,0xb2a8e24a90e5b5f7f4cbd26d350b83674652d65e,0x8bf44c69bb5f28004ed3cb454e9e330ef15a16fe,50000000000000000,21000,1500000007,0,1,0x,,21000,21000,241623,0xB2A8E24a90E5B5F7f4CBD26D350B83674652D65e,sepolia
freq,12,12,2,4,12,5646,22110,69,22112,22155,8916,22155,22155,22155,22155,5646,22155,12,22155,22155


In [8]:
# get the smallest blockNumber
min_block = temp_df["blockNumber"].min()
min_block

'1110074'

In [9]:
# let's rank the people that have contributed to this faucet
addresses = {}
for idx, tx in txs_df.iterrows():
    a_from = tx["from"]
    if a_from not in addresses:
        addresses[a_from] = 0

    value = tx["value"] / 10**18
    addresses[a_from] += value
# turn into df and order by value
addresses_df = pd.DataFrame.from_dict(addresses, orient="index")
addresses_df.columns = ["value"]
addresses_df = addresses_df.sort_values(by="value", ascending=False)
addresses_df.head()

,value
0x1bdae8d8c66badc1d02fe9f58e1586fb00d21b87,1.500000e+08
0x31b98d14007bdee637298086988a0bbd31184523,7.376663e+07
0x9c71fbe2d28080b8afa88cea8a1e319de2c09d44,5.004519e+07
0x8c1e1e5b47980d214965f3bd8ea34c413e120ae4,1.434856e+07
0x1464d4e3c815de3028572b850c6c34dfc57e9320,1.087376e+07


In [10]:
#####################
# get all transactions and add them to df
faucets = {
    "goerli": ["0x4281eCF07378Ee595C564a59048801330f3084eE"],
    "rinkeby": ["0xa7a82DD06901F29aB14AF63faF3358AD101724A8"],
    "kovan": ["0x4281eCF07378Ee595C564a59048801330f3084eE"],
}

# load txs_df from csv
txs_df = pd.read_csv("out/txs_df.csv")
for net in faucets:
    for faucet in faucets[net]:
        try:
            print(f"Getting transactions for {faucet} on {net}")
            txs = get_all_transactions(faucet, net, print_url=False)
            temp_df = pd.DataFrame(txs)
            oldest_block = min(temp_df["blockNumber"])
            newest_block = max(temp_df["blockNumber"])
            print(
                f"Got {len(txs)} transactions for {faucet} on {net}, oldest block {oldest_block}, newest block {newest_block}"
            )

            # etherscan has a 10k transaction limit, so we iteratively get them
            # until we get all transactions
            while oldest_block != 0:
                txs = get_all_transactions(
                    faucet, net, endblock=oldest_block, print_url=False
                )
                # append to temp df
                temp_df = pd.concat([temp_df, pd.DataFrame(txs)])
                # update oldest block
                newest_block = max(temp_df["blockNumber"])
                temp_oldest_block = min(temp_df["blockNumber"])
                print(
                    f"Got {len(txs)} transactions for {faucet} on {net}, oldest block {temp_oldest_block}, newest block {newest_block}"
                )
                if temp_oldest_block == oldest_block:
                    break
                oldest_block = temp_oldest_block

            print(
                f"{faucet} on {net}: block {oldest_block} to {newest_block}, {len(temp_df)} transactions"
            )

            temp_df["faucet"] = faucet
            temp_df["net"] = net
            # add to main df
            txs_df = pd.concat([txs_df, temp_df])

            # save
            temp_df.to_csv(f"out/{faucet}-{net}.csv", index=False)
            txs_df.to_csv(f"out/txs_df.csv", index=False)
        except:
            # try again
            print(f"Error getting transactions for {faucet} on {net}, trying again")
            time.sleep(1)
            continue

txs_df["value"] = txs_df["value"].astype(np.float128)

/tmp/ipykernel_8680/1771399969.py:9: DtypeWarning: Columns (9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  txs_df = pd.read_csv("out/txs_df.csv")


Getting transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7084732, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7081748, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7081596, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7081326, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7081037, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7080870, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a59048801330f3084eE on goerli, oldest block 7080756, newest block 7265783
Got 10000 transactions for 0x4281eCF07378Ee595C564a590

: 

: 